# COMP47670 Assignment 2
## Kate Kiernan
#### 14324876

### 1. Select three review categories of your choice. Scrape all reviews for each category and store them as three separate datasets. For each review, you should store the review text and a class label (i.e. whether the review is “positive” or “negative”).

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

The categories I have chosen are Cafes; Hair and Salons; and Restaurants. The first step is to go through each of the categories and get the URL for the individual businesses in each. 

In [2]:
#Create the variables cafes, salons, and restaurants as the links to the list of businesses in each category
cafes = "http://mlg.ucd.ie/modules/yalp/cafes_list.html"
cafe_res = requests.get(cafes)
cafe_page = str(BeautifulSoup(cafe_res.content))

salons = "http://mlg.ucd.ie/modules/yalp/hair_salons_list.html"
salon_res = requests.get(salons)
salon_page = str(BeautifulSoup(salon_res.content))

restaurants = "http://mlg.ucd.ie/modules/yalp/restaurants_list.html"
restaurant_res = requests.get(restaurants)
restaurant_page = str(BeautifulSoup(restaurant_res.content))

In [3]:
#This function looks parses the html code to find the individual link for each business
def getURL(page):
    start_link = page.find("a href")
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

In [4]:
#Create empty lists to fill with all the business URLs
cafe_url = []
salon_url = []
restaurant_url = []

Within a for loop, parse the html and create a list of URLs for each category.

In [5]:
#Cafes
for i in range(len(cafe_page)):
    url, n = getURL(cafe_page)
    cafe_page = cafe_page[n:]
    if url:
        cafe_url.append(url) #Add to list
    else:
        break

for i in range(len(cafe_url)):
    cafe_url[i] = 'http://mlg.ucd.ie/modules/yalp/' + cafe_url[i]

In [6]:
#Repeat for salons
for i in range(len(salon_page)):
    url, n = getURL(salon_page)
    salon_page = salon_page[n:]
    if url:
        salon_url.append(url)
    else:
        break

for i in range(len(salon_url)):
    salon_url[i] = 'http://mlg.ucd.ie/modules/yalp/' + salon_url[i]

In [7]:
#Repeat for restaurants
for i in range(len(restaurant_page)):
    url, n = getURL(restaurant_page)
    restaurant_page = restaurant_page[n:]
    if url:
        restaurant_url.append(url)
    else:
        break

for i in range(len(restaurant_url)):
    restaurant_url[i] = 'http://mlg.ucd.ie/modules/yalp/' + restaurant_url[i]

The next step is to extract the ratings and review text from each review. The function getReviews will parse the html for each business and store all of the ratings and reviews in a dataframe. For reviews with 3 stars or less, it will store the rating as 'Negative'. For reviews with 4 or 5 stars, it will store the review as 'Postive'.

In [8]:
def getReviews(url):
    ratings = []
    reviews = []
    cafe = []
    
    res = urllib.request.urlopen(url)
    html = res.read().decode()
    
    parser = BeautifulSoup(html, 'html.parser')
    
    #Select the review rating
    for rating in parser.find_all(class_ = "rating"):
        rate = int(rating.find('img').get('alt').split('-')[0])
        
        #Determine if review is positive or negative
        if rate > 3:
            rate = 'Positive'
        else:
            rate = 'Negative'
          
        ratings.append(rate)
    
    #Select the review text and save it
    for review in parser.find_all(class_ = "review-text"):
        text = review.get('')
        reviews.append(review.contents[0])
        
    return(pd.DataFrame({'Rating':ratings, 'Review':reviews}))

In [9]:
#Create empty dataframes for the three categories
cafe_df = pd.DataFrame()
salon_df = pd.DataFrame()
restaurant_df = pd.DataFrame()

In [10]:
#Create the dataframe of cafe reviews
for url in cafe_url:
    getReviews(url)
    
    cafe_df = cafe_df.append(getReviews(url))
    
#Save the dataframe to a csv file
cafe_df.to_csv(r'C:\Users\kateh\Documents\DS\Cafe.csv')

In [11]:
#Repeat for salons
for url in salon_url:
    getReviews(url)
    
    salon_df = salon_df.append(getReviews(url))
    
salon_df.to_csv(r'C:\Users\kateh\Documents\DS\Hair.csv')

In [12]:
#Repeat for restaurants
for url in restaurant_url:
    getReviews(url)
    
    restaurant_df = restaurant_df.append(getReviews(url))
    
restaurant_df.to_csv(r'C:\Users\kateh\Documents\DS\Food.csv')

View the first few elements of each data frame.

In [13]:
cafe_df.head()

,Rating,Review
0,Positive,Pros: Lots of items you would not expect from ...
1,Positive,Best egg-tarts in town! There's really not muc...
2,Negative,I've been to ABC Bakery a few times since I re...
3,Negative,"FYI, Closed Monday's New ownership for about 1..."
4,Positive,The inside may not look like much but they mak...


In [14]:
salon_df.head()

,Rating,Review
0,Positive,"One of the best barbershops I've been to, with..."
1,Positive,Took my son in for a haircut. Barber was great...
2,Negative,"Walked in, said hi. The only barber in there d..."
3,Negative,I came here 10 minutes before 9am to get a hai...
4,Positive,"Great haircut. No fuss no muss, I asked for la..."


In [15]:
restaurant_df.head()

,Rating,Review
0,Negative,My husband and I had a rare afternoon off so w...
1,Positive,For years I thought this was only a wine store...
2,Positive,This place is so charming! I went with my husb...
3,Positive,We have been wanting to try this place for a c...
4,Negative,Decor looks ok but layout is too busy. Difficu...


Check the dimensions of the three data frames.

In [16]:
cafe_df.shape

(2000, 2)

In [17]:
salon_df.shape

(2000, 2)

In [18]:
restaurant_df.shape

(2000, 2)

Check if there is any missing data in the three data frames.

In [19]:
cafe_df.isnull().sum()

Rating    0
Review    0
dtype: int64

In [20]:
salon_df.isnull().sum()

Rating    0
Review    0
dtype: int64

In [21]:
restaurant_df.isnull().sum()

Rating    0
Review    0
dtype: int64

Count the number of positive and negative reviews for each category.

In [22]:
cafe_df.groupby('Rating')['Rating'].count()

Rating
Negative     538
Positive    1462
Name: Rating, dtype: int64

In [23]:
salon_df.groupby('Rating')['Rating'].count()

Rating
Negative     442
Positive    1558
Name: Rating, dtype: int64

In [24]:
restaurant_df.groupby('Rating')['Rating'].count()

Rating
Negative     838
Positive    1162
Name: Rating, dtype: int64

## 2. (a) From the reviews in this category, apply appropriate preprocessing steps to create a numeric representation of the data, suitable for classification.

The following steps will be implemented to preprocess the dataframes:

1. Tokenisation: Split the reviews into individual tokens (words only).
2. Stemming: Shorten words to their roots.
3. Stop word filtering: Remove all stop words (like "and" and "the").
4. Rejoin: Rejoin the meaningful stemmed words into a single string.

Create tokenisation function:

In [25]:
def tokenisation(row):
    review = row['Review']
    tokens = nltk.word_tokenize(review)
    
    #Take words only - no punctuation
    token_words = [w for w in tokens if w.isalpha()]
    
    return token_words

In [26]:
#Apply tokenisaion function to each dataframe
cafe_df['Review'] = cafe_df.apply(tokenisation, axis=1)
salon_df['Review'] = salon_df.apply(tokenisation, axis=1)
restaurant_df['Review'] = restaurant_df.apply(tokenisation, axis=1)

Create stemming function:

In [27]:
stemming = PorterStemmer()

def stem_list(row):
    combined_df = row['Review']
    stemmed_list = [stemming.stem(word) for word in combined_df]
    return (stemmed_list)

In [29]:
#Apply stemming function to each dataframe
cafe_df['Review'] = cafe_df.apply(stem_list, axis=1)
salon_df['Review'] = salon_df.apply(stem_list, axis=1)
restaurant_df['Review'] = restaurant_df.apply(stem_list, axis=1)

Create function to remove stop words:

In [30]:
stops = set(stopwords.words("english"))

def remove_stops(row):
    combined_df = row['Review']
    meaningful_words = [w for w in combined_df if not w in stops]
    
    return (meaningful_words)

In [31]:
#Apply stop word function to each dataframe
cafe_df['Review'] = cafe_df.apply(remove_stops, axis=1)
salon_df['Review'] = salon_df.apply(remove_stops, axis=1)
restaurant_df['Review'] = restaurant_df.apply(remove_stops, axis=1)

Create function to rejoin tokenised/stemmed words into a single string:

In [32]:
def rejoin_words(row):
    combined_df = row['Review']
    joined_words = ( " ".join(combined_df))
    return joined_words

In [33]:
#Apply rejoin words function to each dataframe
cafe_df['Review'] = cafe_df.apply(rejoin_words, axis=1)
salon_df['Review'] = salon_df.apply(rejoin_words, axis=1)
restaurant_df['Review'] = restaurant_df.apply(rejoin_words, axis=1)

View the first few elements of the preprocessed dataframes:

In [34]:
cafe_df.head()

,Rating,Review
0,Positive,pro lot item would expect chine bakeri friendl...
1,Positive,best town realli much say hong kong style fast...
2,Negative,I abc bakeri time sinc I read one top place eg...
3,Negative,fyi close monday new ownership week new menu c...
4,Positive,insid may look like much make mean egg tart ve...


In [35]:
salon_df.head()

,Rating,Review
0,Positive,one best barbershop I great price I honestli c...
1,Positive,took son haircut barber wa great exactli I wan...
2,Negative,walk said hi onli barber courtesi say hi back ...
3,Negative,I came minut befor get haircut open saturday a...
4,Positive,great haircut No fuss muss I ask layer V shape...


In [36]:
restaurant_df.head()

,Rating,Review
0,Negative,My husband I rare afternoon decid tri thi plac...
1,Positive,year I thought thi wa onli wine store sister s...
2,Positive,thi place charm I went husband love simpl clea...
3,Positive,We want tri thi place coupl year final stop la...
4,Negative,decor look ok layout busi difficult walk breat...


## 2. (b) Build a classification model to distinguish between “positive” and “negative” reviews using one of the following classifiers: 
## Naive Bayes, Logistic Regression, Random Forests

I am going to build a Naive Bayes classification model to determine whether a review is positive or negative, starting with Cafes.

In [37]:
#Generate a weighted review-term matrix using TfidfVectorizer
vectorizer = TfidfVectorizer()
cafe_vec = vectorizer.fit_transform(cafe_df['Review'])
cafe_terms = vectorizer.get_feature_names()

#Create a dataframe of the weighted terms
cafe_vec_df = pd.DataFrame(cafe_vec.todense(), columns=cafe_terms)

#Add another column that lists the ratings
cafe_vec_df['Rating'] = list(cafe_df['Rating'])
cafe_vec_df

,aaa,aaand,aau,ab,abbot,abc,aber,aberr,abhorr,abil,...,époustouflé,équilibr,établiss,étaient,était,étoil,étrangement,été,üppige,Rating
0,0.0,0.0,0.0,0.0,0.0,0.181586,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
2,0.0,0.0,0.0,0.0,0.0,0.082311,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
1996,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
1997,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
1998,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative


Select a random sample to train the model and test with all the Cafe data:

In [38]:
cafe_data_train, cafe_data_test, cafe_target_train, cafe_target_test = train_test_split(cafe_vec_df.iloc[:,0:-1], cafe_vec_df.iloc[:,-1], test_size=0.2)

In [39]:
cafe_model = MultinomialNB()
cafe_model.fit(cafe_data_train, cafe_target_train)
cafe_predicted = cafe_model.predict(cafe_data_test)

Repeat all the above steps for Hair and Salons:

In [40]:
#Generate a weighted review-term matrix using TfidfVectorizer
salon_vec = vectorizer.fit_transform(salon_df['Review'])
salon_terms = vectorizer.get_feature_names()

#Create a dataframe of the weighted terms
salon_vec_df = pd.DataFrame(salon_vec.todense(), columns=salon_terms)

#Add another column that lists the ratings
salon_vec_df['Rating'] = list(salon_df['Rating'])
salon_vec_df

,aamaazi,aaron,aback,abbi,abe,abil,abl,aboard,abomin,abort,...,zoe,zone,zooey,zuckerberg,écout,élevé,émili,étaient,été,Rating
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive


In [41]:
salon_data_train, salon_data_test, salon_target_train, salon_target_test = train_test_split(salon_vec_df.iloc[:,0:-1], salon_vec_df.iloc[:,-1], test_size=0.2)

In [42]:
salon_model = MultinomialNB()
salon_model.fit(salon_data_train, salon_target_train)
salon_predicted = salon_model.predict(salon_data_test)

Repeat all the above steps for Restaurants:

In [43]:
#Generate a weighted review-term matrix using TfidfVectorizer
restaurant_vec = vectorizer.fit_transform(restaurant_df['Review'])
restaurant_terms = vectorizer.get_feature_names()

#Create a dataframe of the weighted terms
restaurant_vec_df = pd.DataFrame(restaurant_vec.todense(), columns=restaurant_terms)

#Add another column that lists the ratings
restaurant_vec_df['Rating'] = list(restaurant_df['Rating'])
restaurant_vec_df

,aaah,aaahhhh,aarp,ab,abandon,abc,abdul,abdullah,abend,aber,...,élevé,étaient,était,être,über,übersichtlich,überzeugt,überzogen,übrigen,Rating
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Negative
1996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive
1998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Positive


In [44]:
restaurant_data_train, restaurant_data_test, restaurant_target_train, restaurant_target_test = train_test_split(restaurant_vec_df.iloc[:,0:-1], restaurant_vec_df.iloc[:,-1], test_size=0.2)

In [45]:
restaurant_model = MultinomialNB()
restaurant_model.fit(restaurant_data_train, restaurant_target_train)
restaurant_predicted = restaurant_model.predict(restaurant_data_test)

## 2. (c) Test the predictions of the classification model using an appropriate evaluation strategy. Report and discuss the evaluation results in your notebook.

In [50]:
cafe_model_crossval_score =  cross_val_score(cafe_model, cafe_data_train, cafe_target_train, cv=10, scoring='accuracy')
print("Accuracy of the Naive Bayes model with Cafe data: %0.2f (+/- %0.2f)" % (cafe_model_crossval_score.mean(), cafe_model_crossval_score.std() * 2))

Mean cross-validation accuracy of the Naive Bayes model: 0.74
Accuracy of the Naive Bayes model with Cafe data: 0.74 (+/- 0.02)


In [53]:
salon_model_crossval_score =  cross_val_score(salon_model, salon_data_train, salon_target_train, cv=10, scoring='accuracy')
print("Accuracy of the Naive Bayes model with Salon data: %0.2f (+/- %0.2f)" % (salon_model_crossval_score.mean(), salon_model_crossval_score.std() * 2))

Accuracy of the Naive Bayes model with Salon data: 0.78 (+/- 0.01)


In [54]:
restaurant_model_crossval_score =  cross_val_score(restaurant_model, restaurant_data_train, restaurant_target_train, cv=10, scoring='accuracy')
print("Accuracy of the Naive Bayes model with Restaurant data: %0.2f (+/- %0.2f)" % (restaurant_model_crossval_score.mean(), restaurant_model_crossval_score.std() * 2))

Accuracy of the Naive Bayes model with Restaurant data: 0.81 (+/- 0.04)


The model was fit to each dataframe 10 times, with the data split differently each time. The mean value for the accuracy and the 95% confidence level of each dataframe were calculated. This model is most accurate with the Restaurant data (81% +/- 4%), least accurate with the Cafe data (74% +/- 2%), with the Salon data in between (78% +/- 1%).

## 3. Evaluate the performance of each of your three classification models when applied to data from the other two selected categories:

First train the classifcation model on the Cafe data, and evaluate its performance on the Salon and Restaurant data:

In [58]:
c_trained_on_s =  cross_val_score(cafe_model, salon_data_train, salon_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on cafe data and tested on salon data: %.2f' % c_trained_on_s.mean())

c_trained_on_r =  cross_val_score(cafe_model, restaurant_data_train, restaurant_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on cafe data and tested on restaurant data: %.2f' % c_trained_on_r.mean())

Accuracy of NB model trained on cafe data and tested on salon data: 0.78
Accuracy of NB model trained on cafe data and tested on restaurant data: 0.81


Next train the classifcation model on the Salon data, and evaluate its performance on the Cafe and Restaurant data:

In [59]:
s_trained_on_c =  cross_val_score(salon_model, cafe_data_train, cafe_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on salon data and tested on cafe data: %.2f' % s_trained_on_c.mean())

s_trained_on_r =  cross_val_score(salon_model, restaurant_data_train, restaurant_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on salon data and tested on restaurant data: %.2f' % s_trained_on_r.mean())

Accuracy of NB model trained on salon data and tested on cafe data: 0.74
Accuracy of NB model trained on salon data and tested on restaurant data: 0.81


Finally train the classifcation model on the Restaurant data, and evaluate its performance on the Cafe and Salon data:

In [60]:
r_trained_on_c =  cross_val_score(restaurant_model, cafe_data_train, cafe_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on restaurant data and tested on cafe data: %.2f' % r_trained_on_c.mean())

r_trained_on_s =  cross_val_score(restaurant_model, salon_data_train, salon_target_train, cv=10, scoring='accuracy')
print('Accuracy of NB model trained on resaurant data and tested on salon data: %.2f' % r_trained_on_s.mean())

Accuracy of NB model trained on restaurant data and tested on cafe data: 0.74
Accuracy of NB model trained on resaurant data and tested on salon data: 0.78


The accuracies of the models are slightly different depending on the data the are trained and tested on. This is most likely due to the variations in words that would be used depending on the type of business being reviewed.

However, all of the models are quite accurate, with the lowest score being 74%.